### Analyze the calibrate mode results for TRACERx patients

In [4]:
import os
import pickle
import gzip
from metient.util.globals import *
from metient.lib.vertex_labeling import *

REPO_DIR = os.path.join(os.getcwd(), "../../")


CUDA GPU: False


In [23]:
PARAMS = "calibrate_12202023"
METIENT_OUTPUT_DIR = os.path.join(REPO_DIR, 'metient/data/tracerx_nsclc/metient_outputs')
CONIPHER_OUTPUT_DIR = os.path.join(METIENT_OUTPUT_DIR, f'pyclone_conipher_{PARAMS}', 'calibrate')

#### Which patients have 2 or more mets?

In [24]:
patients = set()
for file in os.listdir(CONIPHER_OUTPUT_DIR):
    if ".pkl.gz" in file:
        name = file.split(".")[0].replace("_calibrate", "")
        patients.add(name)
print(len(patients))

two_plus_met_patients = set()
for patient in patients:
    with gzip.open(os.path.join(CONIPHER_OUTPUT_DIR, f"{patient}.pkl.gz"),'rb') as f:
        pckl = pickle.load(f)
    sites = pckl[OUT_SITES_KEY]
    if len(sites) > 3:
        two_plus_met_patients.add(patient)
print(len(two_plus_met_patients))
two_plus_met_patients

128
21


{'CRUK0010_primary_SU_T1',
 'CRUK0013_primary_SU_T1',
 'CRUK0029_primary_SU_T1',
 'CRUK0035_primary_SU_T1',
 'CRUK0090_primary_SU_T1',
 'CRUK0245_primary_SU_T1',
 'CRUK0311_primary_SU_T1',
 'CRUK0467_primary_SU_T1',
 'CRUK0484_primary_SU_T1',
 'CRUK0487_primary_SU_T1',
 'CRUK0530_primary_SU_T1',
 'CRUK0543_primary_SU_T1',
 'CRUK0559_primary_SU_T1',
 'CRUK0590_primary_SU_T1',
 'CRUK0620_primary_SU_T1',
 'CRUK0620_primary_SU_T2',
 'CRUK0722_primary_SU_T1',
 'CRUK0745_primary_SU_T1',
 'CRUK0748_primary_SU_T1',
 'CRUK0762_primary_SU_T1',
 'CRUK0810_primary_SU_T1'}

#### Which of these patients have their best calibrated solution s.t. lowest m is not the best?

In [25]:
import torch
non_pss_patients = set()
for patient in two_plus_met_patients:
    with gzip.open(os.path.join(CONIPHER_OUTPUT_DIR, f"{patient}.pkl.gz"),'rb') as f:
        pckl = pickle.load(f)
    Vs = pckl[OUT_LABElING_KEY]
    As = pckl[OUT_ADJ_KEY]
    best_soln_m = None
    for i, (V,A) in enumerate(zip(Vs, As)):
        V = torch.tensor(V)
        A = torch.tensor(A)
        m,c,s,_,_ = get_ancestral_labeling_metrics(V.reshape(1, V.shape[0], V.shape[1]), A, None, None, None)
        if i == 0:
            best_soln_m = int(m)
        elif m < best_soln_m:
            non_pss_patients.add(patient)
            print("best_soln_m", best_soln_m)
            print(int(m), int(c), int(s))
non_pss_patients

best_soln_m 8
7 6 2
best_soln_m 8
7 6 3
best_soln_m 8
7 6 3
best_soln_m 8
7 6 3
best_soln_m 8
7 6 3
best_soln_m 9
8 6 2
best_soln_m 9
8 6 2
best_soln_m 8
7 4 2
best_soln_m 21
19 9 4
best_soln_m 21
17 10 4
best_soln_m 21
19 10 3
best_soln_m 19
17 9 4
best_soln_m 19
18 9 3
best_soln_m 19
16 10 4
best_soln_m 10
8 6 3
best_soln_m 10
8 6 3
best_soln_m 10
8 6 3
best_soln_m 10
9 6 3
best_soln_m 10
9 6 3
best_soln_m 10
9 6 3
best_soln_m 10
9 6 3
best_soln_m 10
9 6 3
best_soln_m 10
9 6 3
best_soln_m 10
9 6 3
best_soln_m 10
8 7 3
best_soln_m 10
8 7 3
best_soln_m 10
9 7 2


{'CRUK0035_primary_SU_T1',
 'CRUK0090_primary_SU_T1',
 'CRUK0484_primary_SU_T1',
 'CRUK0487_primary_SU_T1',
 'CRUK0590_primary_SU_T1',
 'CRUK0620_primary_SU_T2'}